# **\[FRAUD\]** 데이터정리 시도(df50 X범주_auto)

김보람  
2023-10-11

# imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import networkx as nx
import sklearn
import xgboost as xgb

# sklearn
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# gnn
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv


# autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/libpyg.so: undefined symbol: _ZN2at4_ops12split_Tensor4callERKNS_6TensorEN3c106SymIntEl
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/torch_scatter/_scatter_cuda.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/coco/anaconda3/en

In [2]:
def down_sample_textbook(df):
    df_majority = df[df.is_fraud==0].copy()
    df_minority = df[df.is_fraud==1].copy()
    df_maj_dowsampled = sklearn.utils.resample(df_majority, n_samples=len(df_minority), replace=False, random_state=42)
    df_downsampled = pd.concat([df_minority, df_maj_dowsampled])
    return df_downsampled

def compute_time_difference(group):
    n = len(group)
    result = []
    for i in range(n):
        for j in range(n):
            time_difference = abs(group.iloc[i].trans_date_trans_time.value - group.iloc[j].trans_date_trans_time.value)
            result.append([group.iloc[i].name, group.iloc[j].name, time_difference])
    return result


class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [3]:
fraudTrain = pd.read_csv("~/Desktop/fraudTrain.csv").iloc[:,1:]

In [4]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

## 데이터정리

In [5]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

In [6]:
df50 = down_sample_textbook(df02)
df50.shape

In [7]:
df50 = df50.reset_index()

In [8]:
N = len(df50)

------------------------------------------------------------------------

# autogluon1: amt

In [9]:
df50 = df50[["amt","is_fraud"]]
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

## A. 데이터

In [10]:
tr = TabularDataset(df50_tr)
tst = TabularDataset(df50_test)

## B. predictor 생성

In [11]:
predictr = TabularPredictor("is_fraud")

No path specified. Models will be saved in: "AutogluonModels/ag-20231011_125519/"

## C.적합(fit)

In [12]:
predictr.fit(tr, presets='best_quality')

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231011_125519/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   746.18 GB / 982.82 GB (75.9%)
Train Data Rows:    9009
Train Data Columns: 1
Label Column: is_fraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
    2 unique label values:  [1, 0]
    If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preproce

In [13]:
predictr.leaderboard()

                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L2   0.894772       0.020864   4.666441                0.009758           2.256641            2       True         14
1           CatBoost_BAG_L1   0.894661       0.004361   1.564344                0.004361           1.564344            1       True          7
2            XGBoost_BAG_L1   0.894439       0.022174   0.569362                0.022174           0.569362            1       True         11
3      LightGBMLarge_BAG_L1   0.894106       0.006745   0.845457                0.006745           0.845457            1       True         13
4           LightGBM_BAG_L1   0.893995       0.016963   0.637937                0.016963           0.637937            1       True          4
5     NeuralNetTorch_BAG_L1   0.888778       0.089444  18.408010                0.089444          18.408010            1       True         12

------------------------------------------------------------------------

# autogluon2: amt, distace

In [22]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

In [23]:
df50 = down_sample_textbook(df02)
df50.shape

In [24]:
df50 = df50.reset_index()

In [25]:
df50['trans_date_trans_time'] = pd.to_datetime(df50['trans_date_trans_time'])
df50['trans_date_trans_time'] = (df50['trans_date_trans_time'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

category_map = {category: index for index, category in enumerate(df50['category'].unique())}
df50['category'] = df50['category'].map(category_map)

def haversine(lat1, lon1, lat2, lon2):
    # 지구의 반지름 (미터)
    radius = 6371.0

    # 라디안으로 변환
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    # Haversine 공식 계산
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = radius * c

    return distance

customer_lat = df50['lat']
customer_lon = df50['long']
store_lat = df50['merch_lat']
store_lon = df50['merch_long']
distances = haversine(customer_lat, customer_lon, store_lat, store_lon)
df50['distance_km'] = distances


In [26]:
df50 = df50[["amt","distance_km", "is_fraud"]]

In [27]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

## A. 데이터

In [28]:
tr = TabularDataset(df50_tr)
tst = TabularDataset(df50_test)

## B. predictor 생성

In [29]:
predictr = TabularPredictor("is_fraud")

No path specified. Models will be saved in: "AutogluonModels/ag-20231011_125738/"

## C.적합(fit)

In [30]:
predictr.fit(tr, presets='best_quality')

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231011_125738/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   738.25 GB / 982.82 GB (75.1%)
Train Data Rows:    9009
Train Data Columns: 2
Label Column: is_fraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
    2 unique label values:  [1, 0]
    If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preproce

In [31]:
predictr.leaderboard()

                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0            XGBoost_BAG_L1   0.895549       0.026780   0.716487                0.026780           0.716487            1       True         11
1       WeightedEnsemble_L2   0.895549       0.036584   3.040059                0.009804           2.323572            2       True         14
2           CatBoost_BAG_L1   0.894772       0.007168   2.017300                0.007168           2.017300            1       True          7
3           LightGBM_BAG_L1   0.893995       0.009826   0.656309                0.009826           0.656309            1       True          4
4         LightGBMXT_BAG_L1   0.889777       0.153301   0.774388                0.153301           0.774388            1       True          3
5     NeuralNetTorch_BAG_L1   0.886891       0.067880  14.708995                0.067880          14.708995            1       True         12

------------------------------------------------------------------------

# autogluon3: amt, time, distace

In [32]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

In [33]:
df50 = down_sample_textbook(df02)
df50.shape

In [34]:
df50 = df50.reset_index()

In [35]:
df50['trans_date_trans_time'] = pd.to_datetime(df50['trans_date_trans_time'])
df50['trans_date_trans_time'] = (df50['trans_date_trans_time'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

category_map = {category: index for index, category in enumerate(df50['category'].unique())}
df50['category'] = df50['category'].map(category_map)

def haversine(lat1, lon1, lat2, lon2):
    # 지구의 반지름 (미터)
    radius = 6371.0

    # 라디안으로 변환
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    # Haversine 공식 계산
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = radius * c

    return distance

customer_lat = df50['lat']
customer_lon = df50['long']
store_lat = df50['merch_lat']
store_lon = df50['merch_long']
distances = haversine(customer_lat, customer_lon, store_lat, store_lon)
df50['distance_km'] = distances


In [36]:
df50 = df50[["amt",'trans_date_trans_time', 'distance_km', "is_fraud"]]

In [37]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

## A. 데이터

In [38]:
tr = TabularDataset(df50_tr)
tst = TabularDataset(df50_test)

## B. predictor 생성

In [39]:
predictr = TabularPredictor("is_fraud")

No path specified. Models will be saved in: "AutogluonModels/ag-20231011_125833/"

## C.적합(fit)

In [40]:
predictr.fit(tr, presets='best_quality')

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231011_125833/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   738.82 GB / 982.82 GB (75.2%)
Train Data Rows:    9009
Train Data Columns: 3
Label Column: is_fraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
    2 unique label values:  [1, 0]
    If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preproce

In [41]:
predictr.leaderboard()

                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L2   0.921967       1.031116  26.060251                0.009939           2.203469            2       True         14
1           LightGBM_BAG_L1   0.900322       0.031546   0.788781                0.031546           0.788781            1       True          4
2           CatBoost_BAG_L1   0.899323       0.005555   2.631501                0.005555           2.631501            1       True          7
3            XGBoost_BAG_L1   0.899323       0.037676   0.793630                0.037676           0.793630            1       True         11
4      LightGBMLarge_BAG_L1   0.892774       0.034059   1.360121                0.034059           1.360121            1       True         13
5         LightGBMXT_BAG_L1   0.891775       0.173170   1.136529                0.173170           1.136529            1       True          3